In [1]:
import gc
import pandas as pd
import numpy as np
import os
import lightgbm as lgb
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
import time

In [2]:
DATA_PATH = "../"
csv_path = os.path.join(DATA_PATH, "atec_anti_fraud_train.csv")
data = pd.read_csv(csv_path)

In [3]:
data.loc[data['label'] == -1, 'label'] = 1

In [4]:
data.loc[data['label'] == 1, 'label'] = 2
data.loc[data['label'] == 0, 'label'] = 1
data.loc[data['label'] == 2, 'label'] = 0

In [5]:
csv_path = os.path.join(DATA_PATH, "atec_anti_fraud_test_b.csv")
data_test = pd.read_csv(csv_path)

In [6]:
data.fillna(0, inplace=True)
data_test.fillna(0, inplace=True)

In [7]:
data_train = data[data['date'] <= 20171021]
data_val = data[data['date'] > 20171021]

In [8]:
del data
gc.collect()

28

In [9]:
X_train = data_train.loc[:,'f1':].as_matrix()
y_train = data_train.loc[:,'label'].as_matrix()

X_val = data_val.loc[:,'f1':].as_matrix()
y_val = data_val.loc[:,'label'].as_matrix()

X_test = data_test.loc[:,'f1':].as_matrix()

In [10]:
del data_train, data_val
gc.collect()

58

In [11]:
res = data_test[['id']]

In [25]:
def eval_func(y_pred, y_true):
    fpr, tpr, thresholds = metrics.roc_curve(y_pred, y_true, pos_label=1)
    score = 0.4 * tpr[np.argwhere(fpr < 0.001)[-1]] + 0.3 * tpr[np.argwhere(
        fpr < 0.005)[-1]] + 0.3 * tpr[np.argwhere(fpr < 0.01)[-1]]
    return  score[0]

In [44]:
def rf_model(train_x, train_y, val_x, val_y, res, test):
#     for i in range(13, 16, 2):
    for i in [15]:
        start_time = time.time()
        clf = RandomForestClassifier(
            n_estimators=25, 
            max_depth=i, 
            criterion='entropy', 
            max_features=i,
            min_samples_split=500,
            min_samples_leaf=1,
#             max_leaf_nodes=i,
            random_state=0, 
            n_jobs=4, 
            class_weight={0: 1, 1: 1})
        clf.fit(train_x, train_y)
        train_proba = clf.predict_proba(train_x)[:, 0]
        val_proba = clf.predict_proba(val_x)[:, 0]
        train_score = eval_func(1-train_y, train_proba)
        val_score = eval_func(1-val_y, val_proba)
        
        cost_time = time.time() - start_time
        
        print('n_estimators = ', i, ' train score : ', train_score, ' validation score : ', val_score, 'diff : ', train_score - val_score, 'cost time : ', cost_time)
    
    res['score'] = clf.predict_proba(test)[:,0]
#     res['score'] = res['score'].apply(lambda x: float('%.6f' % x))
    res.to_csv('submission.csv', index=False)
    return clf

In [45]:
clf = rf_model(X_train, y_train, X_val, y_val, res, X_test)

n_estimators =  15  train score :  0.4397484575225439  validation score :  0.3923900118906064 diff :  0.047358445631937474 cost time :  89.938236951828


C:\software\anaconda\lib\site-packages\ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
